In [1]:
import numpy as np
import pandas as pd

import optuna

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

from sklearn.preprocessing import OrdinalEncoder


In [3]:
df = pd.read_csv("../../dataset/kaggle_ventilator/train_5folds.csv")
test = pd.read_csv("../../dataset/kaggle_ventilator//test.csv")

In [5]:
def add_features(df, transform="robust"):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4)
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    df['u_in_lag10'] = df.groupby('breath_id')['u_in'].shift(10)
    df['u_out_lag10'] = df.groupby('breath_id')['u_out'].shift(10)
    df['u_in_lag_back10'] = df.groupby('breath_id')['u_in'].shift(-10)
    df['u_out_lag_back10'] = df.groupby('breath_id')['u_out'].shift(-10)
    df = df.fillna(0)
    
    df['breath_id__u_in__max'] = df.groupby(['breath_id'])['u_in'].transform('max')
    df['breath_id__u_out__max'] = df.groupby(['breath_id'])['u_out'].transform('max')
    
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['breath_id__u_in__diffmax'] = df.groupby(['breath_id'])['u_in'].transform('max') - df['u_in']
    df['breath_id__u_in__diffmean'] = df.groupby(['breath_id'])['u_in'].transform('mean') - df['u_in']
    
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    df['cross']= df['u_in']*df['u_out']
    df['cross2']= df['time_step']*df['u_out']
    
    r_map = {5: 0, 20: 1, 50: 2}
    c_map = {10: 0, 20: 1, 50: 2}
    df['R'] = df['R'].map(r_map)
    df['C'] = df['C'].map(c_map)
    
    return df


In [6]:
train = add_features(df)
test = add_features(test)

In [7]:
not_used = ["id", "breath_id", "R", "C", "RC", "pressure", "kfold"]
used_columns = list(set(train.columns) - set(not_used))
sorted(used_columns)

['area',
 'time_step',
 'u_in',
 'u_in_cumsum',
 'u_in_lag1',
 'u_in_lag10',
 'u_in_lag2',
 'u_in_lag3',
 'u_in_lag4',
 'u_in_lag_back1',
 'u_in_lag_back10',
 'u_in_lag_back2',
 'u_in_lag_back3',
 'u_in_lag_back4',
 'u_out',
 'u_out_lag1',
 'u_out_lag10',
 'u_out_lag2',
 'u_out_lag3',
 'u_out_lag4',
 'u_out_lag_back1',
 'u_out_lag_back10',
 'u_out_lag_back2',
 'u_out_lag_back3',
 'u_out_lag_back4']

In [8]:
RS = RobustScaler()
train[used_columns] = RS.fit_transform(train[used_columns])
test[used_columns] = RS.transform(test[used_columns])

In [9]:
train.to_csv("../../dataset/kaggle_ventilator/train_5folds_nb7_robust.csv", index=False)


FileNotFoundError: [Errno 2] No such file or directory: '../../dataset/kaggle_ventilator/data/test_nb7_robust.csv'

In [10]:
train.head()

,id,breath_id,R,C,time_step,u_in,u_out,pressure,kfold,area,...,u_in_lag_back3,u_out_lag_back3,u_in_lag4,u_out_lag4,u_in_lag_back4,u_out_lag_back4,u_in_lag10,u_out_lag10,u_in_lag_back10,u_out_lag_back10
0,1,1,1,2,-0.989052,-0.937384,-1.0,5.837492,4,-0.516581,...,3.743915,-1.0,-0.794152,-1.0,4.277824,-1.0,-0.594266,0.0,4.778007,-1.0
1,2,1,1,2,-0.963608,3.049278,-1.0,5.907794,4,-0.514031,...,4.256189,-1.0,-0.794152,-1.0,4.661047,-1.0,-0.594266,0.0,4.756926,-1.0
2,3,1,1,2,-0.938006,3.948195,-1.0,7.876254,4,-0.507768,...,4.639136,-1.0,-0.794152,-1.0,4.634403,-1.0,-0.594266,0.0,5.006451,-1.0
3,4,1,1,2,-0.912278,4.013452,-1.0,11.742872,4,-0.498222,...,4.612511,-1.0,-0.794152,-1.0,4.570046,-1.0,-0.594266,0.0,4.762421,-1.0
4,5,1,1,2,-0.886409,4.568332,-1.0,12.234987,4,-0.484036,...,4.548200,-1.0,-0.777379,-1.0,4.782785,-1.0,-0.594266,0.0,4.528033,-1.0


In [11]:
test.to_csv("../../dataset/kaggle_ventilator/test_nb7_robust.csv", index=False)